<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-27 13:00:47
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.37 C
-------------------
Today PnL: -96.59 K (-0.7%)
Current PnL: -37.19 L (-23.25%)
CY Booked + Current PnL: -21.51 L (-13.45%)
-------------------
Total profit:  1.40 L
Total loss:  -38.59 L
-------------------
Total Booked + Current PnL: 5.47 L (4.11%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.42%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.04 C (75.64%)
Deployed:  1.33 C
Current:  1.37 C
CAGR/XIRR %: 1.6%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-1.16,6.60,10.38,17.67,16649.0,9934.0,160395.0,478.51,27.69,58.0,L-LC,11.10,185.0,0.60,1.19,28.76,XY25,ATH,MINING
50,MASFIN,-4.12,-0.51,22.18,21.57,21622.0,-495.0,97485.0,397.04,-14.18,47.0,H-SC,6.57,167.0,-0.02,0.72,42.26,XR,ATH,FINANCE
77,TTKPRESTIG,-0.64,-31.91,46.99,0.09,32246.0,-32153.0,68624.0,770.00,59.23,31.0,M-SC,11.60,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
51,MEDANTA,-1.56,-4.51,29.94,24.08,36294.0,-5728.0,121222.0,1486.00,-5.20,50.0,X-SC,7.97,87.0,-0.16,0.90,11.03,XY24,NTT,HEALTHCARE
37,ICICIGI,-2.31,0.72,18.48,19.34,37248.0,1450.0,201559.0,2252.93,-15.18,48.0,X-MC,1.09,66.0,0.04,1.50,16.78,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,FINCABLES,6.20,17.47,76.99,107.91,133533.0,25798.0,173442.0,1641.55,5.53,76.0,M-SC,7.05,220.0,0.19,1.29,30.49,OX40N,ATH,CABLES
24,DIXON,4.17,-26.94,79.65,31.26,134296.0,-62160.0,168608.0,18931.72,-67.33,40.0,X-MC,24.11,54.0,-0.46,1.25,4.17,X40N,ATH,IT
69,SURYODAY,3.31,-25.95,66.15,23.03,87715.0,-46471.0,132600.0,216.00,44.74,51.0,H-SC,5.56,166.0,-0.53,0.98,31.57,XR,NTT,BANKS
49,LAOPALA,1.82,-37.77,123.50,39.09,100765.0,-49514.0,81591.0,464.00,78.62,58.0,H-SC,13.40,154.0,-0.49,0.61,9.11,AR,NTT,CERAMICS
61,ROUTE,1.52,-61.30,334.53,68.16,172266.0,-81571.0,51495.0,2215.45,-68.61,31.0,H-SC,30.68,141.0,-0.47,0.38,3.81,SR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-4.12,-0.51,22.18,21.57,21622.0,-495.0,97485.0,397.04,-14.18,47.0,H-SC,6.57,167.0,-0.02,0.72,42.26,XR,ATH,FINANCE
5,ANGELONE,-4.06,-90.32,1182.45,24.20,349556.0,-275688.0,29562.0,3033.00,-85.87,36.0,X-SC,2.46,97.0,-0.79,0.22,23.81,X40N,NTT,FINANCE
38,ICICIPRULI,-3.64,7.56,21.54,30.73,45643.0,14895.0,211900.0,790.00,-21.38,48.0,X-MC,4.13,78.0,0.33,1.57,21.29,X40,ATH,INSURANCE
17,CAMS,-3.49,-10.39,38.91,24.48,87815.0,-26169.0,225687.0,950.00,-14.13,35.0,X-SC,5.65,89.0,-0.30,1.67,10.89,X40N,NTT,MISC
11,BANDHANBNK,-3.41,-13.53,122.53,92.42,295132.0,-37694.0,240865.0,400.00,106.90,67.0,H-SC,8.05,174.0,-0.13,1.79,32.69,XY24,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-4.12,-0.51,22.18,21.57,21622.0,-495.0,97485.0,397.04,-14.18,47.0,H-SC,6.57,167.0,-0.02,0.72,42.26,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,FINCABLES,6.20,17.47,76.99,107.91,133533.0,25798.0,173442.0,1641.55,5.53,76.0,M-SC,7.05,220.0,0.19,1.29,30.49,OX40N,ATH,CABLES
66,SIS,-0.43,-30.96,75.65,21.26,57988.0,-34379.0,76653.0,528.00,1799.48,38.0,H-SC,5.60,168.0,-0.59,0.57,5.67,OX40N,NTT,MISC
77,TTKPRESTIG,-0.64,-31.91,46.99,0.09,32246.0,-32153.0,68624.0,770.00,59.23,31.0,M-SC,11.60,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
41,INDIGOPNTS,-0.66,-33.14,49.60,0.02,57888.0,-57850.0,116709.0,1408.00,78.01,27.0,M-SC,12.18,222.0,-1.00,0.87,1.52,OX40N,NTT,PAINTS
48,KANSAINER,-0.07,-33.16,69.78,13.47,125767.0,-89433.0,180234.0,340.00,-42.48,32.0,H-SC,13.07,162.0,-0.71,1.34,1.32,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1.07,22.13,18.89,45.2,44234.0,42423.0,234165.0,1856.0,39.84,65.0,X-MC,9.08,79.0,0.96,1.74,30.43,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-1.16,6.60,10.38,17.67,16649.0,9934.0,160395.0,478.51,27.69,58.0,L-LC,11.10,185.0,0.60,1.19,28.76,XY25,ATH,MINING
27,FINCABLES,6.20,17.47,76.99,107.91,133533.0,25798.0,173442.0,1641.55,5.53,76.0,M-SC,7.05,220.0,0.19,1.29,30.49,OX40N,ATH,CABLES
1,ABB,-0.76,11.67,30.35,45.57,88668.0,30533.0,292152.0,7934.00,-28.26,66.0,H-MC,8.81,125.0,0.34,2.17,29.73,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.76,11.67,30.35,45.57,88668.0,30533.0,292152.0,7934.00,-28.26,66.0,H-MC,8.81,125.0,0.34,2.17,29.73,AR,NTT,ELECTRICAL
27,FINCABLES,6.20,17.47,76.99,107.91,133533.0,25798.0,173442.0,1641.55,5.53,76.0,M-SC,7.05,220.0,0.19,1.29,30.49,OX40N,ATH,CABLES
50,MASFIN,-4.12,-0.51,22.18,21.57,21622.0,-495.0,97485.0,397.04,-14.18,47.0,H-SC,6.57,167.0,-0.02,0.72,42.26,XR,ATH,FINANCE
85,ZYDUSLIFE,-1.86,-2.87,39.64,35.63,80688.0,-6022.0,203552.0,1286.17,-15.32,55.0,H-MC,4.89,120.0,-0.07,1.51,13.32,AR,ATH,PHARMA
40,INDIAMART,-1.27,-7.48,123.44,106.72,140856.0,-9227.0,114109.0,4810.62,-58.20,41.0,H-SC,8.09,140.0,-0.07,0.85,14.15,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,-0.16,-38.71,156.48,57.19,312771.0,-126248.0,199879.0,485.00,-64.88,18.0,X-SC,18.83,81.0,-0.40,1.48,0.00,XY24,NTT,FMCG
43,INFY,1.20,-11.19,51.16,34.24,142163.0,-35008.0,277880.0,1972.00,-29.97,25.0,X-LC,8.69,3.0,-0.25,2.06,2.28,X40,NTT,IT
12,BATAINDIA,-0.54,-49.01,165.18,35.20,108363.0,-63067.0,65603.0,2096.00,-13.94,28.0,X-SC,22.32,91.0,-0.58,0.49,0.00,X40,NTT,FOOTWEAR
34,HCLTECH,1.25,-9.75,37.21,23.83,81244.0,-23584.0,218340.0,1908.19,-5.29,30.0,X-LC,6.06,8.0,-0.29,1.62,4.75,X40,ATH,IT
53,PGHH,-0.03,-14.20,53.86,32.01,98672.0,-30320.0,183200.0,17616.87,-42.03,31.0,X-MC,9.58,57.0,-0.31,1.36,0.06,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIGI,-2.31,0.72,18.48,19.34,37248.0,1450.0,201559.0,2252.93,-15.18,48.0,X-MC,1.09,66.0,0.04,1.50,16.78,X40,ATH,INSURANCE
78,UNITDSPR,-0.53,1.48,19.00,20.77,45147.0,3474.0,237618.0,1644.00,-10.02,48.0,X-MC,1.28,64.0,0.08,1.76,8.99,X40N,NTT,BREWERIES
22,DABUR,-1.03,1.48,41.60,43.70,104723.0,3671.0,251739.0,735.00,-7.83,54.0,X-MC,1.48,70.0,0.04,1.87,16.75,XY24,BTT,FMCG
59,RELIANCE,-0.13,-9.81,38.93,25.30,70559.0,-19718.0,181245.0,1952.00,-12.34,42.0,X-LC,2.29,21.0,-0.28,1.34,21.41,XY25,BTT,REFINERIES
5,ANGELONE,-4.06,-90.32,1182.45,24.20,349556.0,-275688.0,29562.0,3033.00,-85.87,36.0,X-SC,2.46,97.0,-0.79,0.22,23.81,X40N,NTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
2,ABBOTINDIA,-0.38,-11.92,34.05,18.07,53639.0,-21325.0,157530.0,35195.0,-25.06,37.0,X-MC,11.50,58.0,-0.40,1.17,0.0,X40,ATH,PHARMA
3,ACC,-1.53,-33.24,145.89,64.15,231746.0,-79101.0,158850.0,3906.0,-49.40,32.0,X-SC,9.52,85.0,-0.34,1.18,0.0,XY24,BTT,CEMENT
9,BAJAJHFL,-0.82,-25.60,108.98,55.47,197248.0,-62291.0,180995.0,181.5,-36.26,32.0,X-MC,17.21,60.0,-0.32,1.34,0.0,X40N,ATH,FINANCE
8,AWL,-0.16,-38.71,156.48,57.19,312771.0,-126248.0,199879.0,485.0,-64.88,18.0,X-SC,18.83,81.0,-0.40,1.48,0.0,XY24,NTT,FMCG
12,BATAINDIA,-0.54,-49.01,165.18,35.20,108363.0,-63067.0,65603.0,2096.0,-13.94,28.0,X-SC,22.32,91.0,-0.58,0.49,0.0,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,-4.06,-90.32,1182.45,24.20,349556.0,-275688.0,29562.0,3033.0,-85.87,36.0,X-SC,2.46,97.0,-0.79,0.22,23.81,X40N,NTT,FINANCE
54,QUESS,-1.34,-34.57,118.31,42.84,50322.0,-22472.0,42534.0,424.0,-57.02,37.0,X-SC,17.39,80.0,-0.45,0.32,2.31,XY24,NTT,MISC
58,RELAXO,-0.75,-54.48,234.57,52.31,155009.0,-79078.0,66082.0,1176.0,-51.51,36.0,X-SC,14.78,92.0,-0.51,0.49,0.00,X40N,NTT,FOOTWEAR
12,BATAINDIA,-0.54,-49.01,165.18,35.20,108363.0,-63067.0,65603.0,2096.0,-13.94,28.0,X-SC,22.32,91.0,-0.58,0.49,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,-1.56,-4.51,29.94,24.08,36294.0,-5728.0,121222.0,1486.0,-5.20,50.0,X-SC,7.97,87.0,-0.16,0.90,11.03,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,ITC,-0.91,-16.92,43.31,19.06,122393.0,-57568.0,282598.0,452.00,-54.31,40.0,X-LC,17.19,1.0,-0.47,2.10,2.32,X40,NTT,FMCG
73,TCS,-0.17,-24.24,63.13,23.58,206905.0,-104885.0,327744.0,4311.59,-34.45,31.0,X-LC,9.13,2.0,-0.51,2.43,2.70,X40,ATH,IT
43,INFY,1.20,-11.19,51.16,34.24,142163.0,-35008.0,277880.0,1972.00,-29.97,25.0,X-LC,8.69,3.0,-0.25,2.06,2.28,X40,NTT,IT
75,TMPV,-1.05,-13.90,54.86,33.33,153890.0,-45286.0,280514.0,600.00,-81.44,62.0,X-LC,3.94,4.0,-0.29,2.08,14.68,XY24,NTT,AUTO
34,HCLTECH,1.25,-9.75,37.21,23.83,81244.0,-23584.0,218340.0,1908.19,-5.29,30.0,X-LC,6.06,8.0,-0.29,1.62,4.75,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INDUSINDBK,-0.72,-29.30,87.96,32.89,48014.0,-22620.0,54586.0,1800.00,-357.00,64.0,L-MC,12.10,258.0,-0.47,0.40,50.33,XR,NTT,BANKS
67,SONACOMS,-2.58,-8.09,51.33,39.09,47725.0,-8182.0,92977.0,804.02,-27.22,57.0,M-MC,10.79,193.0,-0.17,0.69,31.53,AR,ATH,AUTO
19,COALINDIA,-1.16,6.60,10.38,17.67,16649.0,9934.0,160395.0,478.51,27.69,58.0,L-LC,11.10,185.0,0.60,1.19,28.76,XY25,ATH,MINING
65,SIEMENS,0.79,-9.64,38.90,25.51,65414.0,-17935.0,168160.0,4671.50,41.28,64.0,H-LC,3.51,51.0,-0.27,1.25,23.92,AR,ATH,ELECTRICAL
7,ATULAUTO,-0.89,-14.05,69.36,45.57,117523.0,-27693.0,169439.0,844.00,3672.97,59.0,M-SC,6.69,248.0,-0.24,1.26,28.23,XY24,NTT,AUTO


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,EASEMYTRIP,-0.69,-52.43,206.26,45.70,173494.0,-92701.0,84114.0,26.40,-12.72,61.0,M-SC,18.19,204.0,-0.53,0.62,38.59,XY24,NTT,TRAVEL
27,FINCABLES,6.20,17.47,76.99,107.91,133533.0,25798.0,173442.0,1641.55,5.53,76.0,M-SC,7.05,220.0,0.19,1.29,30.49,OX40N,ATH,CABLES
1,ABB,-0.76,11.67,30.35,45.57,88668.0,30533.0,292152.0,7934.00,-28.26,66.0,H-MC,8.81,125.0,0.34,2.17,29.73,AR,NTT,ELECTRICAL
7,ATULAUTO,-0.89,-14.05,69.36,45.57,117523.0,-27693.0,169439.0,844.00,3672.97,59.0,M-SC,6.69,248.0,-0.24,1.26,28.23,XY24,NTT,AUTO
11,BANDHANBNK,-3.41,-13.53,122.53,92.42,295132.0,-37694.0,240865.0,400.00,106.90,67.0,H-SC,8.05,174.0,-0.13,1.79,32.69,XY24,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.81
1,25,45.83
2,50,77.88


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.95
MC    30.78
LC    25.30
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.51
X40      24.22
X40N     12.96
AR        9.34
XY25      9.12
XR        8.65
OX40N     7.38
SR        0.85
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.16
H-SC    22.67
X-LC    21.82
M-SC    11.02
X-SC     9.54
H-MC     5.10
M-MC     2.12
H-LC     1.25
L-LC     1.19
M-LC     1.04
L-SC     0.72
L-MC     0.40
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.98,-14.65,53.43
IT,11.61,-39.98,110.14
FINANCE,11.21,-38.09,92.42
MISC,6.94,-38.97,95.82
ELECTRICAL,6.52,-6.83,45.81
PAINTS,5.13,-33.65,53.51
INSURANCE,4.96,-1.54,36.28
PHARMA,4.21,-3.50,36.08
AUTO,4.03,-14.95,58.78


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3419975.0,21
X40,1456535.0,15
X40N,1374841.0,10
XR,1324082.0,12
AR,1313548.0,10
OX40N,799608.0,10
XY25,388279.0,6
SR,304005.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3737859.0,24
X-SC,1518459.0,10
M-SC,1432957.0,14
X-MC,1391845.0,15
X-LC,1331517.0,12
H-MC,392219.0,3
L-SC,193797.0,2
M-LC,151875.0,1
M-MC,100268.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1265844.0      6
           AR         955406.0      5
           XR         848292.0      7
M-SC       XY24       839910.0      6
X-LC       X40        708958.0      6
X-SC       X40N       702005.0      4
           XY24       631133.0      4
X-MC       X40        562256.0      7
           X40N       442707.0      4
H-SC       OX40N      364312.0      4
M-SC       OX40N      336981.0      5
H-SC       SR         304005.0      2
X-LC       XY24       265010.0      2
           X40N       230129.0      2
H-MC       XY24       222863.0      1
X-MC       XY24       195215.0      2
           XY25       191667.0      2
X-SC       X40        185321.0      2
M-SC       XR         180419.0      2
H-MC       AR         169356.0      2
M-LC       XR         151875.0      1
X-LC       XY25       127420.0      2
L-SC       OX40N       98315.0      1
           XR          95482.0      1
M-SC       AR          75647.0      1
H-LC       AR          65414.0      1
M-MC       XY25        52543.0      1
L-MC       XR          48014.0      1
M-MC       AR          47725.0      1
L-LC       XY25        16649.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
